#expert version


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)

from tensorflow.keras import datasets 


2.2.0


In [ ]:
input_shape = (28,28,1)
num_classes = 10

In [ ]:
inputs = layers.Input(input_shape, dtype=tf.float64)

net = layers.Conv2D(32,(3,3),padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,(3,3),padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D(pool_size = (2,2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Conv2D(64,(3,3),padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,(3,3),padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D(pool_size = (2,2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs=net,name='Basic_CNN')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
#차원 추가
x_train = x_train[...,tf.newaxis]
x_test = x_test[...,tf.newaxis]
#정규화
x_train,x_test = x_train/255.0 , x_test/255.0

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train))#데이터를 넣은거임, 부가적으로 정보넣어주면 알아서 반응
train_ds = train_ds.shuffle(1000)#buffer size
train_ds = train_ds.batch(32)#batch size 결정

test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test))#데이터를 넣은거임, 부가적으로 정보넣어주면 알아서 반응
test_ds = test_ds.batch(32)#batch size 결정

In [ ]:
#시각화
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy')
#model.fit(train_ds,epochs = 10000)
#keras 식이다

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [ ]:
#loss계산을 계속 보여주는게 아니라 평균값을 통해 보여주려함
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'train_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

#Training

In [ ]:
#이렇게하면  학습이 시작되면 돌아가도록함
@tf.function
def train_step(images,labels):
  with tf.GradientTape() as tape:
    predictions = model(images)#train mode variable을 바꿀 수 없다. test는 dropout없다.
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [ ]:
for epoch in range(2):
  for images, labels in train_ds :
    train_step(images, labels)

  for test_images, test_labels in test_ds :
    test_step(test_images,test_labels)

  template = 'Epoch {}, Loss : {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1, train_loss.result(),train_accuracy.result()*100,test_loss.result(),test_accuracy.result()*100))

Epoch 1, Loss : 0.10089141130447388, Accuracy: 96.92166137695312, Test Loss: 0.06378097087144852, Test Accuracy: 97.64999389648438
Epoch 2, Loss : 0.06731808930635452, Accuracy: 97.94667053222656, Test Loss: 0.047881465405225754, Test Accuracy: 98.33499908447266
